# GEREKLİ KÜTÜPHANELERİ TANIMLA

In [2]:
import os
import glob
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

import urllib.request
import random

# SEÇİLMİŞ KATEGORİLER İÇİN VERİ SETİNİ İNDİR

In [3]:
CLASSES = ['airplane', 'alarm_clock', 'apple', 'banana', 'bandage', 'basketball', 'bird', 'bowtie', 'bread', 'broom', 'butterfly', 'candle', 'car', 'cat', 'cell_phone', 'circle', 'clock', 'crown', 'cup', 'diamond', 'door', 'envelope', 'eyeglasses', 'fish', 'fork', 'flower', 'hand', 'headphones', 'ladder', 'key', 'line', 'moustache', 'mountain', 'moon', 'mushroom', 'pear', 'power_outlet', 'shorts', 'star', 'square', 'tent', 'television', 'toothbrush', 'triangle', 't-shirt', 'umbrella']
def download():
  base = 'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/'
  for c in CLASSES:
    cls_url = c.replace('_', '%20')
    path = base+cls_url+'.npy'
    print(path)
    urllib.request.urlretrieve(path, 'data/'+c+'.npy')

!mkdir data
download()

https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/airplane.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/alarm%20clock.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/apple.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/banana.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bandage.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/basketball.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bird.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bowtie.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/bread.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/broom.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/butterfly.npy
https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/candle.npy
https://storage.googleapis.com/quickdraw_dataset/ful

# VERİLERİ YÜKLE
Her kategori .npy formatında çeşitli sayıda array örneklerinden oluşmaktadır. RAM sınırlı bir kaynak olduğu için bu örneklem sayısını (MAX_ITEMS_PER_CLASS sınırlı tutmamız gerekiyor.

In [5]:
# Seçilen kategorilerin sayısı
NUMBER_OF_CLASSES = len(CLASSES)

# Her kategoriden en fazla kaç örnek resim kullanılacak
MAX_ITEMS_PER_CLASS = 10000

def load_data(root, vfold_ratio = 0.2):
  all_files = glob.glob(os.path.join(root, '*.npy'))

  x = np.empty([0,784])
  y = np.empty([0])
  class_names = []

  # Verileri kısım kısım hafızaya al
  for idx, file in enumerate(all_files):
    data = np.load(file)
    data = data[0: MAX_ITEMS_PER_CLASS, :]
    labels = np.full(data.shape[0], idx)

    x = np.concatenate((x, data), axis = 0)
    y = np.append(y, labels)

    class_name, ext = os.path.splitext(os.path.basename(file))
    class_names.append(class_name)

  data = None
  labels = None

  # Veri setini randomize et 
  permutation = np.random.permutation(y.shape[0])
  x = x[permutation, :]
  y = y[permutation]

  # Eğitim ve test veri seti olarak ayır 
  vfold_size = int(x.shape[0] / NUMBER_OF_CLASSES * (vfold_ratio * NUMBER_OF_CLASSES))

  x_test = x[0:vfold_size, :]
  y_test = y[0:vfold_size]

  x_train = x[vfold_size:x.shape[0], :]
  y_train = y[vfold_size:y.shape[0]]
  return x_train, y_train, x_test, y_test, class_names

x_train, y_train, x_test, y_test, class_names = load_data('data')
num_classes = len(class_names)
image_size = 28

# VERİLERİ ÖN İŞLEMDEN GEÇİR

# Yeniden şekillendir ve normalize et
x_train = x_train.reshape(x_train.shape[0], image_size, image_size, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], image_size, image_size, 1).astype('float32')

x_train /= 255.0
x_test /= 255.0

# Sınıf/kategori vektörlerini matrislere çevir
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# MODEL

Tanımla, eğit, değerlendir.

In [6]:
# Modeli tanımla
model = keras.Sequential()
model.add(layers.Convolution2D(16, (3, 3),
                        padding='same',
                        input_shape=x_train.shape[1:], activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(32, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Convolution2D(64, (3, 3), padding='same', activation= 'relu'))
model.add(layers.MaxPooling2D(pool_size =(2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(NUMBER_OF_CLASSES, activation='softmax')) 

# Modeli eğit
adam = tf.optimizers.Adam()
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['top_k_categorical_accuracy'])
print(model.summary())

# Modeli verilen girdilerle eğit
model.fit(x = x_train, 
          y = y_train, 
          validation_split = 0.1, 
          batch_size = 256, 
          verbose = 2,
          epochs = 5)

# Eğitilmiş modeli, eğitimde kullanılmayan verilerle değerlendir.
# Model için kayıp ve ölçüm değerini döndür.
score = model.evaluate(x_test, y_test, verbose = 0)
print('Test accuracy: {:0.2f}%'.format(score[1] * NUMBER_OF_CLASSES))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 576)               0

# MODELİ DÖNÜŞTÜR VE İNDİR

In [ ]:
# Kategorileri dosyaya yaz
with open('class_names.txt', 'w') as file_handler:
    for item in class_names:
        file_handler.write("{}\n".format(item))

# tensorflow.js kütüphanesini yükle
!pip install tensorflowjs

# Modeli kaydet
model.save('keras.h5')

!mkdir model
# Modeli web formatına çevir (tensorflowjs kütüphanesiyle modele erişim imkanı sağla)
!tensorflowjs_converter --input_format keras keras.h5 model/

# Sıkıştır
!cp class_names.txt model/class_names.txt
!zip -r model.zip model

# İndir
from google.colab import files
files.download('model.zip')

Test accuracy: 44.55%
     |████████████████████████████████| 71kB 1.5MB/s 
     |████████████████████████████████| 92kB 3.4MB/s 
     |████████████████████████████████| 137.3MB 24kB/s 
     |████████████████████████████████| 256kB 70.3MB/s 
     |████████████████████████████████| 921kB 76.1MB/s 
  Created wheel for PyInquirer: filename=PyInquirer-1.0.3-cp36-none-any.whl size=32851 sha256=c9fd29e29b16177dbbb166010f367a70635a13978b98411057a280ce25421055
  Stored in directory: /root/.cache/pip/wheels/52/6c/b1/3e4b0e8daf42a92883c7641c0ea8ffb62e0490ebed2faa55ad
Successfully built PyInquirer
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: tensorflow-hub 0.8.0
    Uninstalling tensorflow-hub-0.8.0:
      Successfully uninstalled tensorflow-hub-0.8.0
  Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Found 

  adding: model/ (stored 0%)
  adding: model/class_names.txt (deflated 37%)
  adding: model/group1-shard1of1.bin (deflated 7%)
  adding: model/model.json (deflated 82%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>